# Set up

In [1]:
import sys
sys.path.insert(0, '..')
from libs.dataloaders import mahnob, deap, dreamer
from libs.utils import finetuning
import torch

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-7fk9danh because the default path (/home/dutruong/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

from libs.dataloaders import mahnob, deap, dreamer
from joblib import Parallel, delayed
import xml.etree.ElementTree as ET

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import normalize
from torch.utils.tensorboard import SummaryWriter

import pickle

In [3]:
def get_sessions_subjs():
    path = "/net2/expData/affective_eeg/mahnob_dataset/Sessions"
    sessions = os.listdir(path)
    subjs = {}
    scores = {}
    for session in sessions:
        mytree = ET.parse(f'{path}/{session}/session.xml')
        myroot = mytree.getroot()
        if myroot[0].attrib['id'] not in subjs:
            subjs[myroot[0].attrib['id']] = []
        if 'feltVlnc' in myroot.attrib:
            subjs[myroot[0].attrib['id']].append(session)
            scores[session] = int(myroot.attrib['feltVlnc'])
    return subjs, scores

In [4]:
def subject_cv(dataset, subject_sessions:list, model, nfreezelayers=0.5):
    """
    Run leave one subject out cross-validation using `model`
    Notes:
        - Does _not_ enforce train data balancing, but sets equal priors (assumes data is roughly balanced)
        - Enforce test data; skip fold when per-label count < 3
    @param dataset - dataset
    @param list[list] subject_sessions - sessions binned by subject for leave one subject out
    """

    accs = [] # np.empty((len(subject_sessions))) # n_subjects
    # accs[:] = np.nan
    train_accs = []
    
    for n_subject, sessions in enumerate(tqdm(subject_sessions.values())):
        train_data = [dataset[i] for i in range(len(dataset)) if dataset.sessions[i] not in sessions]
        test_data = [dataset[i] for i in range(len(dataset)) if dataset.sessions[i] in sessions]
        
        Y_train = [train_data[i][1] for i in range(len(train_data))]
        Y_test = [test_data[i][1] for i in range(len(test_data))]

        if len(Y_train) == 0 or len(Y_test) == 0:
            print("Skipping: Y train or Y test having no values")
            continue
        if len(np.unique(Y_test, return_counts=True)[-1]) != len(np.unique(Y_train, return_counts=True)[-1]):
            print("Skipping: num classes not being equivalent in train and test")
            continue  
            
        # balancing train
        u_val, u_count = np.unique(Y_train, return_counts=True)
        u_count = list(u_count)
        if len(u_count) != 2:
            print("Skipping: class count is not exactly 2")
            continue
        max_indices = np.argwhere(np.squeeze(Y_train == np.argsort(u_count)[-1])) # 0 1 classes. Thus class name is also indices
        # np.random.shuffle(max_indices)
        remove_indices = max_indices[:np.abs(np.diff(u_count))[0]]
        train_data_balanced = [train_data[i] for i in range(len(train_data)) if i not in remove_indices]
        Y_train = [train_data_balanced[i][1] for i in range(len(train_data_balanced))]
        u, c = np.unique(Y_train, return_counts=True)
        print(u)
        print(c)
        
        # balancing test
        u, c = np.unique(Y_test, return_counts=True)
        max_indices = np.argwhere(np.squeeze(Y_test == np.argsort(c)[-1])) # 0 1 classes. Thus class name is also indices
        # np.random.shuffle(max_indices)
        remove_indices = max_indices[:np.abs(np.diff(c))[0]]
        test_data_balanced = [test_data[i] for i in range(len(test_data)) if i not in remove_indices]
        Y_test = [test_data_balanced[i][1] for i in range(len(test_data_balanced))]
        u, c = np.unique(Y_test, return_counts=True)
        print(u)
        print(c)
        
        if min(np.unique(Y_test, return_counts=True)[-1]) < 3:
            print("Skipping: fewer than 3 samples per class detected")
            continue
            
        # initialize and train model
        target_model = finetuning.FinetuneModel(model=model, n_freezelayers=nfreezelayers)
        lr = 0.001
        if model == 'alexnet':
            optimizer = torch.optim.Adam(target_model.model.parameters(), lr = lr)
        else:
            optimizer = torch.optim.SGD(target_model.model.parameters(), lr = lr)
        target_model.train(train_data_balanced, log_dir='./runs/mahnob', optimizer=optimizer, lr=lr, epochs=20, early_stopping_eps=0.0001, lr_decay_nepoch=100)

        train_acc = target_model.score(train_data_balanced)
        print('train acc', train_acc)
        train_accs.append(train_acc)
        
        test_acc = target_model.score(test_data_balanced)
        print('test acc', test_acc)
        accs.append(test_acc)
        
    return accs, train_accs

In [9]:
def per_subject_cv(dataset, subject_sessions:list, model, nfreezelayers=0.5):
    """
    Run per subject LOO validation using `model`
    Notes:
        - Enforce train data balancing
        - Enforce test data; skip fold when per-label count < 3 # NOT THE CASE
    @param dataset - DEAP dataset
    @param list subject_sessions - subjects or sessions; used for LOO
    @param fn model - DFT model to use for finetuning
    """
    dataset_seqs = len(dataset)
    dataset_feats = np.prod(np.shape(dataset[0][0][0]))
    pc_sweep_list = np.arange(1, min(dataset_seqs, dataset_feats+1))
    
    # Generate parameter set of parallelization
    results = []
    train_accs = []
    test_accs = []
    for session in tqdm(subject_sessions):
        data_idx = [i for i in range(len(dataset)) if dataset.sessions[i] == session]
        job_list = []
        
        train_accs_cv = []
        test_accs_cv = []
        for k_bootstrap, test_idx in enumerate(tqdm(data_idx)):
            train_idx = [i for i in data_idx if i != test_idx]
            train_data = [dataset[i] for i in train_idx]
            test_data = [dataset[test_idx]]
            
            Y_train = [train_data[i][1] for i in range(len(train_data))]
            Y_test = [test_data[i][1] for i in range(len(test_data))]
            
            # balancing train
            u_val, u_count = np.unique(Y_train, return_counts=True)
            u_count = list(u_count)
            if len(u_count) != 2:
                print("Skipping: class count is not exactly 2")
                continue
            max_indices = np.argwhere(np.squeeze(Y_train == np.argsort(u_count)[-1])) # 0 1 classes. Thus class name is also indices
            # np.random.shuffle(max_indices)
            remove_indices = max_indices[:np.abs(np.diff(u_count))[0]]
            train_data_balanced = [train_data[i] for i in range(len(train_data)) if i not in remove_indices]
            Y_train = [train_data_balanced[i][1] for i in range(len(train_data_balanced))]
            u, c = np.unique(Y_train, return_counts=True)
            print(u)
            print(c)

            # initialize and train model
            target_model = finetuning.FinetuneModel(model=model, n_freezelayers=nfreezelayers)
            lr = 0.0001
            if model == 'alexnet':
                optimizer = torch.optim.Adam(target_model.model.parameters(), lr = lr)
            else:
                optimizer = torch.optim.SGD(target_model.model.parameters(), lr = lr)
            target_model.train(train_data_balanced, log_dir='./runs/deap', optimizer=optimizer, lr=lr, bs=32, epochs=50, early_stopping_eps=0.0001, lr_decay_nepoch=100)
           
            train_acc_cv = target_model.score(train_data_balanced)
            print('train acc', train_acc_cv)
            train_accs_cv.append(train_acc_cv)
            test_acc_cv = target_model.score(test_data)
            print('test acc', test_acc_cv)
            test_accs_cv.append(test_acc_cv)
        
        train_accs.append(train_accs_cv)
        test_accs.append(test_accs_cv)
  
    return np.array(train_accs), np.array(test_accs)



In [10]:
def trainmax2test(train, test):
    '''subject pc bootstrap -> subject bootstrap'''
    ## Max with fewest nPC
    # _train = np.nanmean(train, axis=2) # avg on bootstrap
    # _train = np.nan_to_num(_train, nan=-1.) # hacky way to get around all nan argmax
    # _argmax = np.argmax(_train, axis=1) # find max acc per pc
    # return np.array([subj[_argmax[n_subj], :] for n_subj, subj in enumerate(test)])

    ## Max with most nPC
    # _train = np.nanmean(train, axis=2) # avg on bootstrap
    # _train = np.nan_to_num(_train, nan=-1.) # hacky way to get around all nan argmax
    # _train = np.flip(_train, axis=1)
    # _argmax = np.argmax(_train, axis=1) # find max acc per pc
    # _test = np.flip(test, axis=1)
    # _test = np.array([subj[_argmax[n_subj], :] for n_subj, subj in enumerate(_test)])
    # _test = np.flip(_test, axis=1)
    # return _test

    ## Max with most nPC
    _train = np.nan_to_num(train, nan=-1.) # hacky way to get around all nan argmax
    _train = np.flip(_train, axis=1)
    _argmax = np.argmax(_train, axis=1) # find max acc per pc
    _test = np.flip(test, axis=1)
    return np.array([[k[_argmax[n_subj, n_k]] for n_k, k in enumerate(subj.T)] for n_subj, subj in enumerate(_test)])

def plot_overview(train, test, ev, dataset=None, dataset_clip=10):
    mdl_accs = trainmax2test(train, test)

    if dataset is not None:
        print(f"Clipping datasets with fewer than {dataset_clip} samples...")
        valid_sessions_idx = np.argwhere(np.unique(dataset.sessions, return_counts=True)[1] >= 10).squeeze()
    else:
        valid_sessions_idx = np.arange(mdl_accs.shape[0])
    
    mdl_accs = mdl_accs[valid_sessions_idx,:]
    mdl_means = np.nanmean(mdl_accs, axis=1) # avg on bootstrap
    print(np.nanmean(mdl_means))
    mdl_stderr = np.nanstd(mdl_accs, axis=1) / np.sqrt(np.count_nonzero(~np.isnan(mdl_accs), axis=1))
    
    plt.subplot(2,1,1)
    plt.errorbar(np.arange(len(mdl_means)), mdl_means, yerr=mdl_stderr, color='k', ecolor="grey")
    plt.ylabel("Accuracy")

    plt.subplot(2,1,2)
    plt.plot(np.cumsum(ev, axis=-1).T)
    plt.xlabel("Subject")
    plt.ylabel("Exp Var")
    plt.show()

In [11]:
import torch
import torchvision.models as torchmodels
def vgg16_augment(model):
    model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])
def inception_augment(model):
    model = torch.nn.Sequential(*list(torchmodels.inception_v3(weights="DEFAULT").children())[:-3])

# Mahnob Topomap

In [ ]:
dataset = mahnob.MahnobDataset(
    x_params={
        "feature": "TopomapImg",
        "window": -1,
        "stride": -1
    },
    n_sessions=None,
    y_mode='bimodal',
    y_keys=['feltVlnc'],
    seed=49,
)
with open('mahnob.pkl', 'wb') as fout:
    pickle.dump({'dataset': dataset}, fin)

In [ ]:
with open('mahnob.pkl', 'rb') as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects, subject_scores = get_sessions_subjs()
test_accs, train_accs = subject_cv(dataset, subjects, 'vgg16', nfreezelayers=0.5)
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_mahnob_topomap_vgg16.pkl", "wb") as fout:
    pickle.dump({"test_accs": test_accs, 
                 "train_accs": train_accs}, fout)

  0%|          | 0/29 [00:00<?, ?it/s]

[0 1]
[124 124]
[0 1]
[6 6]
epoch 0:
loss=2.086860775947571
weights=-75480.5, -27580.109375
acc=0.4717741935483871

epoch 1:
loss=0.8091582246124744
weights=-75711.59375, -27657.408203125
acc=0.4879032258064516

epoch 2:
loss=0.7241806611418724
weights=-75827.5390625, -27694.3046875
acc=0.5362903225806451

epoch 3:
loss=0.7136886902153492
weights=-75897.4375, -27719.58203125
acc=0.5443548387096774

epoch 4:
loss=0.7127120643854141
weights=-75951.59375, -27734.875
acc=0.5241935483870968

epoch 5:
loss=0.6981852352619171
weights=-76017.875, -27749.82421875
acc=0.5604838709677419

epoch 6:
loss=0.7036980204284191
weights=-76082.0234375, -27773.341796875
acc=0.625

epoch 7:
loss=0.6795616149902344
weights=-76112.734375, -27774.640625
acc=0.5524193548387096

epoch 8:
loss=0.6396292559802532
weights=-76140.765625, -27778.73046875
acc=0.6895161290322581

epoch 9:
loss=0.7238996718078852
weights=-76252.6328125, -27826.181640625
acc=0.5282258064516129

epoch 10:
loss=0.6848902739584446
weights=

  3%|▎         | 1/29 [02:05<58:47, 125.98s/it]

train acc 0.7983870967741935
test acc 0.5
[0 1]
[126 126]
[0 1]
[4 4]
epoch 0:
loss=2.4058537371456623
weights=-75591.65625, -27578.837890625
acc=0.5198412698412699

epoch 1:
loss=0.706218957901001
weights=-75840.7421875, -27653.287109375
acc=0.5119047619047619

epoch 2:
loss=0.7840640768408775
weights=-75955.4453125, -27692.65234375
acc=0.5595238095238095

epoch 3:
loss=0.7165509201586246
weights=-76002.1875, -27701.779296875
acc=0.49603174603174605

epoch 4:
loss=0.7163389697670937
weights=-76066.0703125, -27709.89453125
acc=0.5595238095238095

epoch 5:
loss=0.6807017251849174
weights=-76118.171875, -27716.96484375
acc=0.5595238095238095

epoch 6:
loss=0.7074670940637589
weights=-76196.5, -27731.796875
acc=0.5436507936507936

epoch 7:
loss=0.6996206492185593
weights=-76304.890625, -27750.35546875
acc=0.6111111111111112

epoch 8:
loss=0.6673833541572094
weights=-76411.0234375, -27763.314453125
acc=0.6071428571428571

epoch 9:
loss=0.6727485097944736
weights=-76498.65625, -27774.603515

  7%|▋         | 2/29 [04:15<57:34, 127.94s/it]

...

train acc 0.8185483870967742
test acc 0.5
[0 1]
[126 126]
[0 1]
[4 4]
epoch 0:
loss=1.7339239493012428
weights=-75472.8125, -27601.625
acc=0.49603174603174605

epoch 1:
loss=0.8143098540604115
weights=-75758.3671875, -27708.3671875
acc=0.5396825396825397

epoch 2:
loss=0.7648037187755108
weights=-75877.03125, -27740.453125
acc=0.49603174603174605

epoch 3:
loss=0.7184590250253677
weights=-75934.7578125, -27752.38671875
acc=0.5

epoch 4:
loss=0.6909527778625488
weights=-76000.578125, -27770.505859375
acc=0.5238095238095238

epoch 5:
loss=0.696101900190115
weights=-76048.265625, -27779.466796875
acc=0.626984126984127

epoch 6:
loss=0.6841322109103203
weights=-76059.921875, -27780.138671875
acc=0.5634920634920635

epoch 7:
loss=0.6732783019542694
weights=-76106.6328125, -27799.07421875
acc=0.6468253968253969

epoch 8:
loss=0.6908111870288849
weights=-76132.25, -27809.595703125
acc=0.6468253968253969

epoch 9:
loss=0.643537450581789
weights=-76144.40625, -27812.212890625
acc=0.626984126984

 72%|███████▏  | 21/29 [34:11<16:17, 122.21s/it]

train acc 0.8214285714285714
test acc 0.875
[0 1]
[124 124]
[0 1]
[6 6]
epoch 0:
loss=2.2953626960515976
weights=-75490.046875, -27558.4609375
acc=0.5241935483870968

epoch 1:
loss=0.8432479873299599
weights=-75738.140625, -27645.65234375
acc=0.47580645161290325

epoch 2:
loss=0.7349862400442362
weights=-75857.890625, -27680.115234375
acc=0.5604838709677419

epoch 3:
loss=0.6975569240748882
weights=-75940.078125, -27688.11328125
acc=0.5967741935483871

epoch 4:
loss=0.7043640986084938
weights=-75998.1328125, -27683.888671875
acc=0.5201612903225806

epoch 5:
loss=0.6984967067837715
weights=-76036.4921875, -27678.796875
acc=0.5201612903225806

epoch 6:
loss=0.6998748816549778
weights=-76061.8671875, -27679.9375
acc=0.5362903225806451

epoch 7:
loss=0.6743900068104267
weights=-76119.0546875, -27686.154296875
acc=0.6370967741935484

epoch 8:
loss=0.6654310636222363
weights=-76172.78125, -27695.671875
acc=0.6209677419354839

epoch 9:
loss=0.6502243168652058
weights=-76190.6015625, -27697.95

 76%|███████▌  | 22/29 [36:19<14:26, 123.84s/it]

train acc 0.875
test acc 0.5
[0 1]
[128 128]
[0 1]
[2 2]
Skipping: fewer than 3 samples per class detected
[0 1]
[127 127]
[0 1]
[3 3]
epoch 0:
loss=2.3690596222877502
weights=-75517.359375, -27531.779296875
acc=0.484251968503937

epoch 1:
loss=0.8310687765479088
weights=-75777.3671875, -27617.365234375
acc=0.4921259842519685

epoch 2:
loss=0.7274862416088581
weights=-75889.7265625, -27644.947265625
acc=0.468503937007874

epoch 3:
loss=0.7312106043100357
weights=-76012.953125, -27674.287109375
acc=0.5275590551181102

epoch 4:
loss=0.695473313331604
weights=-76082.359375, -27689.412109375
acc=0.5

epoch 5:
loss=0.7074551470577717
weights=-76134.421875, -27706.390625
acc=0.5393700787401575

epoch 6:
loss=0.6865022107958794
weights=-76160.4921875, -27714.5234375
acc=0.5905511811023622

epoch 7:
loss=0.6859933212399483
weights=-76172.6015625, -27710.9375
acc=0.5275590551181102

epoch 8:
loss=0.6948674023151398
weights=-76237.0390625, -27722.017578125
acc=0.5905511811023622

epoch 9:
loss=0

In [7]:
with open("result_finetune_mahnob_topomap_vgg16.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_accs']
    test_accs = results['test_accs']
print(np.mean(train_accs))
print(np.mean(test_accs))
print(test_accs)

0.7695690571618264
0.5353260869565217
[0.5, 0.5, 0.5, 0.5, 0.5, 0.75, 0.16666666666666666, 0.5, 0.5, 0.75, 0.5, 0.5, 0.625, 0.5, 0.5, 0.875, 0.5, 0.5, 0.5, 0.5, 0.5625, 0.3333333333333333, 0.75]


In [25]:
with open('mahnob.pkl', 'rb') as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects, subject_scores = get_sessions_subjs()
acc, train_acc = subject_cv(dataset, subjects, 'alexnet', nfreezelayers=0.5)
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_mahnob_topomap_alexnet.pkl", "wb") as fout:
    pickle.dump({"acc": acc, 
                 "train_acc": train_acc}, fout)

  0%|          | 0/29 [00:00<?, ?it/s]

[0 1]
[124 124]
[0 1]
[6 6]
epoch 0:
loss=14.139340192079544
weights=-56970.47265625, -57123.4609375
acc=0.5

epoch 1:
loss=7.736098006367683
weights=-57167.921875, -57320.8125
acc=0.5725806451612904

epoch 2:
loss=6.542928129434586
weights=-57332.984375, -57485.80078125
acc=0.5362903225806451

epoch 3:
loss=5.526005148887634
weights=-57461.78125, -57614.5390625
acc=0.5967741935483871

epoch 4:
loss=4.891119122505188
weights=-57592.94921875, -57745.65625
acc=0.5564516129032258

epoch 5:
loss=4.655238978564739
weights=-57720.30078125, -57872.9609375
acc=0.5766129032258065

epoch 6:
loss=3.7546351775527
weights=-57818.9765625, -57971.59375
acc=0.5241935483870968

epoch 7:
loss=3.08794666826725
weights=-57901.35546875, -58053.9375
acc=0.5604838709677419

epoch 8:
loss=2.7656322047114372
weights=-57968.5859375, -58121.12890625
acc=0.5766129032258065

epoch 9:
loss=2.065520592033863
weights=-58041.3984375, -58193.8984375
acc=0.5604838709677419

epoch 10:
loss=2.6545271202921867
weights=-581

  3%|▎         | 1/29 [00:25<11:53, 25.47s/it]

train acc 0.7298387096774194
test acc 0.75
[0 1]
[126 126]
[0 1]
[4 4]
epoch 0:
loss=15.190779730677605
weights=-56887.453125, -57040.140625
acc=0.5158730158730159

epoch 1:
loss=10.610091239213943
weights=-57152.66796875, -57305.23046875
acc=0.5238095238095238

epoch 2:
loss=5.9395797699689865
weights=-57314.81640625, -57467.3125
acc=0.5277777777777778

epoch 3:
loss=5.62483224272728
weights=-57446.0078125, -57598.4609375
acc=0.5277777777777778

epoch 4:
loss=5.185177780687809
weights=-57583.29296875, -57735.703125
acc=0.5515873015873016

epoch 5:
loss=5.562332630157471
weights=-57710.0234375, -57862.390625
acc=0.5476190476190477

epoch 6:
loss=4.311609037220478
weights=-57798.24609375, -57950.5703125
acc=0.5833333333333334

epoch 7:
loss=3.0444581024348736
weights=-57877.2109375, -58029.5
acc=0.5753968253968254

epoch 8:
loss=2.699818432331085
weights=-57961.0, -58113.2578125
acc=0.5952380952380952

epoch 9:
loss=2.6836012825369835
weights=-58058.9921875, -58211.2109375
acc=0.5912698

  7%|▋         | 2/29 [00:51<11:31, 25.61s/it]

...

train acc 0.7336065573770492
test acc 0.625
[0 1]
[127 127]
[0 1]
[3 3]
epoch 0:
loss=9.871069073677063
weights=-56866.6796875, -57022.25
acc=0.547244094488189

epoch 1:
loss=10.050296410918236
weights=-57090.05859375, -57245.5390625
acc=0.5905511811023622

epoch 2:
loss=7.836717918515205
weights=-57276.90234375, -57432.3046875
acc=0.562992125984252

epoch 3:
loss=5.964949056506157
weights=-57445.27734375, -57600.609375
acc=0.5787401574803149

epoch 4:
loss=4.279846616089344
weights=-57576.57421875, -57731.8515625
acc=0.594488188976378

epoch 5:
loss=4.074184224009514
weights=-57673.84765625, -57829.08984375
acc=0.562992125984252

epoch 6:
loss=3.9224721267819405
weights=-57744.8984375, -57900.11328125
acc=0.5669291338582677

epoch 7:
loss=3.317011907696724
weights=-57843.95703125, -57999.140625
acc=0.594488188976378

epoch 8:
loss=5.306240685284138
weights=-57975.9140625, -58131.0546875
acc=0.5590551181102362

epoch 9:
loss=3.8530998453497887
weights=-58103.83984375, -58258.94140625
a

 97%|█████████▋| 28/29 [09:23<00:23, 23.89s/it]

train acc 0.7401574803149606
test acc 0.8333333333333334
[0 1]
[124 124]
[0 1]
[6 6]
epoch 0:
loss=16.17906764149666
weights=-56995.9140625, -57148.953125
acc=0.5

epoch 1:
loss=8.559167385101318
weights=-57234.58984375, -57387.5390625
acc=0.5524193548387096

epoch 2:
loss=7.1790545135736465
weights=-57381.49609375, -57534.37890625
acc=0.5604838709677419

epoch 3:
loss=5.52822171151638
weights=-57488.33203125, -57641.171875
acc=0.5846774193548387

epoch 4:
loss=3.9682788364589214
weights=-57616.06640625, -57768.8671875
acc=0.5604838709677419

epoch 5:
loss=5.503811396658421
weights=-57743.0625, -57895.8203125
acc=0.5362903225806451

epoch 6:
loss=3.3792404532432556
weights=-57871.92578125, -58024.6328125
acc=0.5685483870967742

epoch 7:
loss=2.7524577379226685
weights=-57967.1953125, -58119.8515625
acc=0.5645161290322581

epoch 8:
loss=2.466585136950016
weights=-58035.3671875, -58187.9921875
acc=0.6048387096774194

epoch 9:
loss=1.8100427649915218
weights=-58083.21875, -58235.82421875


100%|██████████| 29/29 [09:49<00:00, 20.33s/it]

train acc 0.6209677419354839
test acc 0.5833333333333334


In [5]:
with open("result_finetune_mahnob_topomap_alexnet.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_acc']
    test_accs = results['acc']
print(np.mean(train_accs))
print(np.mean(test_accs))
print(test_accs)

0.6875171072940942
0.5488095238095239
[0.75, 0.375, 0.5, 0.5, 0.5, 0.5, 0.3333333333333333, 0.75, 0.5, 0.5, 0.625, 0.5, 0.375, 0.5, 0.6, 0.625, 0.5, 0.3333333333333333, 0.7142857142857143, 0.6, 0.625, 0.8333333333333334, 0.5833333333333334]


# Mahnob Spectrogram

In [ ]:
with open('mahnob_spectrogram.pkl', 'rb') as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects, subject_scores = get_sessions_subjs()
test_accs, train_accs = subject_cv(dataset, subjects, 'vgg16')
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_mahnob_specto_vgg16.pkl", "wb") as fout: # lr = 0.001
    pickle.dump({"test_accs": test_accs, 
                 "train_accs": train_accs}, fout)

In [ ]:
with open("result_finetune_mahnob_specto_vgg16.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_accs']
    test_accs = results['test_accs']
print(np.mean([np.mean(i) for i in train_accs]))
print(np.mean([np.mean(i) for i in test_accs]))

# DEAP Topomap

In [ ]:
dataset = deap.DeapDataset(
    x_params={
        "feature": "TopomapImg",
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['arousal'],
    seed=49,
    balanced=False,
)
subjects = list(np.unique(dataset.sessions))
with open('deap_topomap.pkl', 'wb') as fout:
    pickle.dump({'dataset': dataset}, fout)

In [ ]:
with open("deap_topomap.pkl", "rb") as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects = list(np.unique(dataset.sessions))
train_accs, test_accs = per_subject_cv(dataset, subjects, 'vgg16', nfreezelayers=-1)
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_deap_topomap_vgg16_baseline.pkl", "wb") as fout:
    pickle.dump({"acc": test_accs, 
                 "train_acc": train_accs}, fout)

  0%|          | 0/33 [00:00<?, ?it/s]

[0 1]
[15 15]
epoch 0:
loss=7.323601722717285
weights=-74955.828125, -27416.791015625
acc=0.5

epoch 1:
loss=8.209901809692383
weights=-74964.03125, -27420.19140625
acc=0.5

epoch 2:
loss=8.672297477722168
weights=-74972.875, -27423.173828125
acc=0.5

epoch 3:
loss=3.0705223083496094
weights=-74976.0078125, -27424.20703125
acc=0.5

epoch 4:
loss=2.0354802012443542
weights=-74978.015625, -27424.88671875
acc=0.5333333333333333

epoch 5:
loss=1.177109569311142
weights=-74978.7890625, -27425.3125
acc=0.6

epoch 6:
loss=1.0007916986942291
weights=-74979.3046875, -27425.466796875
acc=0.5

epoch 7:
loss=2.0432541370391846
weights=-74981.15625, -27425.9921875
acc=0.5333333333333333

epoch 8:
loss=1.6119381189346313
weights=-74982.453125, -27426.431640625
acc=0.5333333333333333

epoch 9:
loss=0.8634026646614075
weights=-74982.7890625, -27426.673828125
acc=0.5

epoch 10:
loss=1.3735151290893555
weights=-74984.0703125, -27427.23046875
acc=0.6666666666666666

epoch 11:
loss=1.1327230334281921
weig


  3%|▎         | 1/33 [00:29<15:46, 29.57s/it]

train acc 0.6333333333333333
test acc 1.0
[0 1]
[15 15]
epoch 0:
loss=21.371949195861816
weights=-74970.0234375, -27422.4140625
acc=0.5

epoch 1:
loss=3.6356316804885864
weights=-74974.1640625, -27423.720703125
acc=0.5

epoch 2:
loss=3.1605314016342163
weights=-74977.5234375, -27424.9453125
acc=0.5

epoch 3:
loss=2.1463075280189514
weights=-74978.9296875, -27425.6328125
acc=0.5

epoch 4:
loss=2.171322524547577
weights=-74980.9375, -27426.439453125
acc=0.5

epoch 5:
loss=1.0450547337532043
weights=-74981.53125, -27426.703125
acc=0.6666666666666666

epoch 6:
loss=1.4097904860973358
weights=-74982.84375, -27427.125
acc=0.5

epoch 7:
loss=1.6269981861114502
weights=-74984.1796875, -27427.48828125
acc=0.5

epoch 8:
loss=1.2694897055625916
weights=-74985.1796875, -27427.8046875
acc=0.4666666666666667

epoch 9:
loss=0.5941910147666931
weights=-74985.28125, -27427.884765625
acc=0.5333333333333333

epoch 10:
loss=1.4091920852661133
weights=-74986.4140625, -27428.20703125
acc=0.6

epoch 11:
loss


  6%|▌         | 2/33 [00:49<12:28, 24.13s/it]

...

train acc 0.8
test acc 0.0
[0 1]
[5 5]
epoch 0:
loss=5.7528252601623535
weights=-74951.2890625, -27414.98828125
acc=0.5

epoch 1:
loss=4.818234920501709
weights=-74952.953125, -27415.890625
acc=0.5

epoch 2:
loss=23.380706787109375
weights=-74965.09375, -27420.4453125
acc=0.5

epoch 3:
loss=7.401278495788574
weights=-74968.828125, -27421.93359375
acc=0.5

epoch 4:
loss=3.7623558044433594
weights=-74970.515625, -27422.662109375
acc=0.5

epoch 5:
loss=3.1424455642700195
weights=-74972.28125, -27423.20703125
acc=0.5

epoch 6:
loss=2.641709327697754
weights=-74973.7890625, -27423.681640625
acc=0.5

epoch 7:
loss=2.1255431175231934
weights=-74974.75, -27424.05859375
acc=0.5

epoch 8:
loss=1.4445526599884033
weights=-74975.40625, -27424.361328125
acc=0.5

epoch 9:
loss=1.6171433925628662
weights=-74976.171875, -27424.658203125
acc=0.5

epoch 10:
loss=0.6358029246330261
weights=-74976.3515625, -27424.736328125
acc=0.5

epoch 11:
loss=1.077763557434082
weights=-74976.71875, -27424.8125
acc=0.5


 23%|██▎       | 3/13 [00:34<01:56, 11.65s/it]

train acc 0.7
test acc 0.0
[0 1]
[5 5]
epoch 0:
loss=3.7581028938293457
weights=-74951.390625, -27414.6171875
acc=0.5

epoch 1:
loss=19.150466918945312
weights=-74960.3984375, -27418.640625
acc=0.5

epoch 2:
loss=7.502202033996582
weights=-74965.0390625, -27419.96484375
acc=0.5

epoch 3:
loss=3.2814369201660156
weights=-74966.2578125, -27420.47265625
acc=0.5

epoch 4:
loss=4.919559001922607
weights=-74969.15625, -27421.44921875
acc=0.5

epoch 5:
loss=2.4824471473693848
weights=-74970.4609375, -27422.0390625
acc=0.5

epoch 6:
loss=2.309020519256592
weights=-74971.6875, -27422.345703125
acc=0.6

epoch 7:
loss=0.37979111075401306
weights=-74971.5078125, -27422.29296875
acc=0.5

epoch 8:
loss=2.03503155708313
weights=-74972.1796875, -27422.462890625
acc=0.5

epoch 9:
loss=3.356041669845581
weights=-74973.40625, -27423.111328125
acc=0.5

epoch 10:
loss=2.07474422454834
weights=-74974.5859375, -27423.39453125
acc=0.6

epoch 11:
loss=0.8834749460220337
weights=-74974.90625, -27423.578125
acc=


 31%|███       | 4/13 [00:46<01:44, 11.60s/it]

train acc 0.6
test acc 0.0
[0 1]
[6 6]
epoch 0:
loss=10.767579078674316
weights=-74954.65625, -27416.046875
acc=0.5

epoch 1:
loss=3.2399814128875732
weights=-74956.15625, -27416.76171875
acc=0.5

epoch 2:
loss=9.251437187194824
weights=-74961.609375, -27418.611328125
acc=0.5

epoch 3:
loss=10.966933250427246
weights=-74967.2421875, -27420.974609375
acc=0.5

epoch 4:
loss=3.982900619506836
weights=-74969.125, -27421.70703125
acc=0.5

epoch 5:
loss=3.743595838546753
weights=-74971.46875, -27422.66015625
acc=0.5



In [4]:
with open("result_finetune_deap_topomap_vgg16_baseline.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_acc']
    test_accs = results['acc']
print(np.mean([np.mean(i) for i in train_accs]))
print(np.mean([np.mean(i) for i in test_accs]))

0.6110182632755213
0.5313789132542935


In [ ]:
with open("deap_topomap.pkl", "rb") as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects = list(np.unique(dataset.sessions))
train_accs, test_accs = per_subject_cv(dataset, subjects, 'alexnet', nfreezelayers=0.5)
with open("result_finetune_deap_topomap_alexnet.pkl", "wb") as fout:
    pickle.dump({"acc": test_accs, 
                 "train_acc": train_accs}, fout)

In [59]:
with open("result_finetune_deap_topomap_alexnet.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_acc']
    test_accs = results['acc']
print(np.mean([np.mean(i) for i in train_accs]))
print(np.mean([np.mean(i) for i in test_accs]))

0.6680289823728671
0.5428749911469661


# DEAP Spectrogram

In [9]:
dataset = deap.DeapDataset(
    x_params={
        "feature": "SpectrogramImg",
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['arousal'],
    seed=49,
    balanced=False,
)
subjects = list(np.unique(dataset.sessions))
with open('deap_spectra.pkl', 'wb') as fout:
    pickle.dump({'dataset': dataset}, fout)

  0%|          | 0/32 [00:00<?, ?it/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    6.7s finished
  3%|▎         | 1/32 [00:17<09:01, 17.48s/it][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    6.0s finished
  6%|▋         | 2/32 [00:32<08:08, 16.27s/it][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    5.9s finished
  9%|▉         | 3/32 [00:48<07:39, 15.83s/it][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    2.8s
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    6.0s finished


In [ ]:
with open("deap_spectra.pkl", "rb") as fin:
    dataset = pickle.load(fin)
dataset = dataset['dataset']
print(dataset[0][0].shape)
subjects = list(np.unique(dataset.sessions))
train_accs, test_accs = per_subject_cv(dataset, subjects, 'vgg16', nfreezelayers=-1)
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_deap_specto_vgg16_baseline.pkl", "wb") as fout:
    pickle.dump({"acc": test_accs, 
                 "train_acc": train_accs}, fout)

(3, 224, 224)


  0%|          | 0/1056 [00:00<?, ?it/s]

[0 1]
[480 480]
epoch 0:
loss=0.7220591048399607
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 1:
loss=0.7175391773382823
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 2:
loss=0.7334053059418996
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 3:
loss=0.7197903156280517
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 4:
loss=0.7184780816237132
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 5:
loss=0.7259876310825348
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 6:
loss=0.7219635605812073
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 7:
loss=0.7383175055185954
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 8:
loss=0.735087678829829
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 9:
loss=0.734385605653127
weights=-74950.375, -27413.7890625
acc=0.49895833333333334

epoch 10:
loss=0.7378003458182018
weights=-74950.375, -2

In [ ]:
with open("result_finetune_deap_specto_vgg16_baseline.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_acc']
    test_accs = results['acc']
print(np.mean([np.mean(i) for i in train_accs]))
print(np.mean([np.mean(i) for i in test_accs]))

In [ ]:
with open("deap_spectra.pkl", "rb") as fin:
    dataset = pickle.load(fin)
dataset = dataset['dataset']
print(dataset[0][0].shape)
subjects = list(np.unique(dataset.sessions))
train_accs, test_accs = per_subject_cv(dataset, subjects, 'vgg16', nfreezelayers=0.5)
print('train mean', np.mean(train_accs))
print('test mean', np.mean(test_accs))
with open("result_finetune_deap_specto_vgg16.pkl", "wb") as fout:
    pickle.dump({"acc": test_accs, 
                 "train_acc": train_accs}, fout)

In [ ]:
with open("result_finetune_deap_specto_vgg16.pkl", "rb") as fin:
    results = pickle.load(fin)
    train_accs = results['train_acc']
    test_accs = results['acc']
print(np.mean([np.mean(i) for i in train_accs]))
print(np.mean([np.mean(i) for i in test_accs]))

# Deap raw with EEGNet

In [4]:
dataset = deap.DeapDataset(
    x_params={
        "feature": "RawData",
    },
    sessions=None,
    y_mode='bimodal',
    y_keys=['arousal'],
    seed=49,
    balanced=False,
)
subjects = list(np.unique(dataset.sessions))
with open('deap_raw.pkl', 'wb') as fout:
    pickle.dump({'dataset': dataset}, fout)

100%|██████████| 32/32 [00:00<00:00, 581029.13it/s]


In [3]:
with open("deap_raw.pkl", "rb") as fin:
    dataset = pickle.load(fin)
    dataset = dataset['dataset']
subjects = list(np.unique(dataset.sessions))
test_idx = np.random.choice(list(range(len(dataset))), size=int(len(dataset)*0.2), replace=False)

In [4]:
data_idx = list(range(len(dataset)))
train_idx = [i for i in data_idx if i not in test_idx]
train_data = [dataset[i] for i in train_idx]
test_data = dataset[test_idx]

Y_train = [train_data[i][1] for i in range(len(train_data))]
Y_test = [test_data[i][1] for i in range(len(test_data))]

# balancing train
u_val, u_count = np.unique(Y_train, return_counts=True)
u_count = list(u_count)
if len(u_count) != 2:
    print("Skipping: class count is not exactly 2")

max_indices = np.argwhere(np.squeeze(Y_train == np.argsort(u_count)[-1])) # 0 1 classes. Thus class name is also indices
# np.random.shuffle(max_indices)
remove_indices = max_indices[:np.abs(np.diff(u_count))[0]]
train_data_balanced = [train_data[i] for i in range(len(train_data)) if i not in remove_indices]
Y_train = [train_data_balanced[i][1] for i in range(len(train_data_balanced))]
u, c = np.unique(Y_train, return_counts=True)
print(u)
print(c)

# initialize and train model
model = 'eegnet'
target_model = finetuning.FinetuneModel(model='eegnet')
lr = 0.0001
if model == 'vgg16':
    optimizer = torch.optim.SGD(target_model.model.parameters(), lr = lr)
else:
    optimizer = torch.optim.Adam(target_model.model.parameters(), lr = lr)
    
target_model.train(train_data_balanced, log_dir='./runs/eegnet', optimizer=optimizer, lr=lr, epochs=50, early_stopping_eps=0.0001, lr_decay_nepoch=100)

[0 1]
[232 232]


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 0:
loss=0.7011814672371437
weights=22.240097045898438, 24.103242874145508
acc=0.5452586206896551

epoch 1:
loss=0.6848877873913995
weights=22.29717445373535, 24.10740852355957
acc=0.5689655172413793

epoch 2:
loss=0.6773836304401529
weights=23.599964141845703, 24.14621925354004
acc=0.5797413793103449

epoch 3:
loss=0.6582443816908474
weights=24.116159439086914, 24.263137817382812
acc=0.6120689655172413

epoch 4:
loss=0.6475668693410939
weights=24.529462814331055, 24.380014419555664
acc=0.6379310344827587

epoch 5:
loss=0.6362240725550158
weights=24.48312759399414, 24.47767448425293
acc=0.6810344827586207

epoch 6:
loss=0.6270966057119698
weights=24.603515625, 24.600656509399414
acc=0.728448275862069

epoch 7:
loss=0.6185163793892696
weights=24.618614196777344, 24.76158905029297
acc=0.7392241379310345

epoch 8:
loss=0.6103309516249031
weights=24.747472763061523, 24.873924255371094
acc=0.6982758620689655

epoch 9:
loss=0.606857614270572
weights=25.14211654663086, 24.993770599365234

In [6]:
# torch.manual_seed(0)
# np.random.seed(0)
Y_test = test_data[:][1]
u_val, u_count = np.unique(Y_test, return_counts=True)
print(u_val, u_count)

sample, label = test_data
sample = torch.tensor(sample, dtype=torch.float, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')).unsqueeze(1)
print(sample.shape)
target_model.model.eval()
with torch.no_grad():
    pred = target_model.model(sample)
print(sum((torch.argmax(pred, axis=1).numpy(force=True) == np.squeeze(label)))/len(pred))


[0 1] [68 65]
torch.Size([133, 1, 8064, 32])
0.5639097744360902
